# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [4]:
columns = list(df.columns)

In [5]:
# Load the data
df = df.loc[:, columns].copy()

# Drop null columns
df = df.dropna(axis="columns", how="all")

# Drop null rows
df = df.dropna()

# remove issued loans
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

In [6]:
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [7]:
# Create our features
X = df.drop(columns="loan_status")
X = pd.get_dummies(X)

# Create our target
y = df['loan_status']

In [8]:
y.head()

0    low_risk
1    low_risk
2    low_risk
3    low_risk
4    low_risk
Name: loan_status, dtype: object

In [9]:
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
0,10700.0,7.672,52800,0.431818,5,1,22800,0,1,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0,1,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0,0,1
3,10700.0,7.664,52700,0.430740,5,1,22700,0,1,0
4,10800.0,7.698,53000,0.433962,5,1,23000,1,0,0


In [10]:
y.describe()

count        77536
unique           2
top       low_risk
freq         75036
Name: loan_status, dtype: object

In [11]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [12]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_training, X_testing, y_training, y_testing = train_test_split(X, y, random_state=78)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [13]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [14]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_training)

In [15]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_training)
X_test_scaled = X_scaler.transform(X_testing)

# Simple Logistic Regression

In [44]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='newton-cg', random_state=1)
model.fit(X_training, y_training)

LogisticRegression(random_state=1, solver='newton-cg')

In [45]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_testing)
balanced_accuracy_score(y_testing, y_pred)

0.964598594548552

In [46]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_testing, y_pred)

array([[  561,    39],
       [  109, 18675]])

In [47]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_testing, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.94      0.99      0.88      0.96      0.92       600
   low_risk       1.00      0.99      0.94      1.00      0.96      0.94     18784

avg / total       0.99      0.99      0.94      0.99      0.96      0.93     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [48]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_training, y_training)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56252, 'high_risk': 56252})

In [49]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver="newton-cg", random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1, solver='newton-cg')

In [50]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_testing)
balanced_accuracy_score(y_testing, y_pred)

0.9952988358886996

In [51]:
# Display the confusion matrix
confusion_matrix(y_testing, y_pred)

array([[  598,     2],
       [  114, 18670]])

In [52]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_testing, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      1.00      0.99      0.91      1.00      0.99       600
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18784

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



### SMOTE Oversampling

In [53]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_training, y_training)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56252, 'high_risk': 56252})

In [54]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='newton-cg', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1, solver='newton-cg')

In [55]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_testing)
balanced_accuracy_score(y_testing, y_pred)

0.992878691084611

In [56]:
# Display the confusion matrix
confusion_matrix(y_testing, y_pred)

array([[  595,     5],
       [  111, 18673]])

In [57]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_testing, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       600
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18784

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [58]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
centroids = ClusterCentroids(random_state=1)
X_resampled, y_resampled = centroids.fit_resample(X_training, y_training)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 1900, 'low_risk': 1900})

In [59]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='newton-cg', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1, solver='newton-cg')

In [60]:
# Calculate the balanced accuracy score
balanced_accuracy_score(y_testing, y_pred)

0.992878691084611

In [61]:
# Display the confusion matrix
y_pred = model.predict(X_testing)
confusion_matrix(y_testing, y_pred)

array([[  586,    14],
       [  110, 18674]])

In [62]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_testing, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.98      0.99      0.90      0.99      0.97       600
   low_risk       1.00      0.99      0.98      1.00      0.99      0.97     18784

avg / total       0.99      0.99      0.98      0.99      0.99      0.97     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [63]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X,y)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 74069, 'low_risk': 74591})

In [64]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='newton-cg', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1, solver='newton-cg')

In [65]:
# Calculate the balanced accuracy score
balanced_accuracy_score(y_testing, y_pred)

0.9854053094832482

In [66]:
# Display the confusion matrix
y_pred = model.predict(X_testing)
confusion_matrix(y_testing, y_pred)

array([[  598,     2],
       [  120, 18664]])

In [67]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_testing, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      1.00      0.99      0.91      1.00      0.99       600
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18784

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   The Naive Random Oversampling model had the highest balanced accuracy score.  I ran these models using the newton-cg solver.  Which is very similar to the default, but I ended up with slightly different results using the 

2. Which model had the best recall score?

    Naive Random Oversampling and the SMOTEEN Over and Under Sampling models returned perfect recall scores. This was a bit alarming, so I ran the models in the liblinear solver.  

3. Which model had the best geometric mean score?

    Naive Random Oversampling and the SMOTEEN Over and Under Sampling models returned perfect geometric mean scores.   
